In [20]:
import warnings
warnings.filterwarnings('ignore')

In [21]:
import numpy as np
import pandas as pd
import hvplot.pandas
import plotly.express as px
import dataframe_image as dfi
import tensorflow as tf
from pathlib import Path
from collections import Counter

In [22]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from imblearn.metrics import classification_report_imbalanced

In [23]:
car_clean_df = pd.read_csv("Resources/car_clean.csv")
car_clean_df.head()

,Unnamed: 0,manufacturer,model,year,selling_price,fuel,seller_type,mileage
0,0,Maruti,Swift Dzire VDI,2014,450000,Diesel,Individual,23.40
1,1,Skoda,Rapid 1.5 TDI Ambition,2014,370000,Diesel,Individual,21.14
2,2,Honda,City 2017-2020 EXi,2006,158000,Petrol,Individual,17.70
3,3,Hyundai,i20 Sportz Diesel,2010,225000,Diesel,Individual,23.00
4,4,Maruti,Swift VXI BSIII,2007,130000,Petrol,Individual,16.10


In [24]:
# Split our preprocessed data into our features and target arrays
X = pd.get_dummies(car_clean_df.drop(columns="fuel").copy())
y = car_clean_df["fuel"].copy()

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [25]:
# Generate our categorical variable lists
fuel_cat = car_clean_df.dtypes[car_clean_df.dtypes == "object"].index.tolist()

car_clean_df[fuel_cat].nunique()

manufacturer      31
model           1983
fuel               3
seller_type        3
dtype: int64

In [26]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(car_clean_df[fuel_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(fuel_cat)
encode_df.head()

,manufacturer_Ambassador,manufacturer_Ashok,manufacturer_Audi,manufacturer_BMW,manufacturer_Chevrolet,manufacturer_Daewoo,manufacturer_Datsun,manufacturer_Fiat,manufacturer_Force,manufacturer_Ford,...,model_i20 Sportz AT 1.4,model_i20 Sportz Diesel,model_i20 Sportz Option 1.2,model_i20 Sportz Option 1.4 CRDi,fuel_Diesel,fuel_Other,fuel_Petrol,seller_type_Dealer,seller_type_Individual,seller_type_Trustmark Dealer
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0


In [27]:
# Merge one-hot encoded features and drop the originals
car_clean_df = car_clean_df.merge(encode_df,left_index=True, right_index=True)
car_clean_df = car_clean_df.drop(fuel_cat, 1)
car_clean_df.head()

,Unnamed: 0,year,selling_price,mileage,manufacturer_Ambassador,manufacturer_Ashok,manufacturer_Audi,manufacturer_BMW,manufacturer_Chevrolet,manufacturer_Daewoo,...,model_i20 Sportz AT 1.4,model_i20 Sportz Diesel,model_i20 Sportz Option 1.2,model_i20 Sportz Option 1.4 CRDi,fuel_Diesel,fuel_Other,fuel_Petrol,seller_type_Dealer,seller_type_Individual,seller_type_Trustmark Dealer
0,0,2014,450000,23.40,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,1,2014,370000,21.14,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,2,2006,158000,17.70,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3,3,2010,225000,23.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,4,2007,130000,16.10,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0


In [28]:
# Split our preprocessed data into our features and target arrays
y = car_clean_df["fuel"].values
X = car_clean_df.drop(["fuel"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

KeyError: 'fuel'

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

ValueError: could not convert string to float: 'Tata'

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 80)                474480    
                                                                 
 dense_4 (Dense)             (None, 30)                2430      
                                                                 
 dense_5 (Dense)             (None, 1)                 31        
                                                                 
Total params: 476,941
Trainable params: 476,941
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch', 
    period=5)

In [ ]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

Epoch 1/100


ValueError: in user code:

    File "C:\Users\Alvin Tran\anaconda3\envs\mlenv\lib\site-packages\keras\engine\training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\Alvin Tran\anaconda3\envs\mlenv\lib\site-packages\keras\engine\training.py", line 1010, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\Alvin Tran\anaconda3\envs\mlenv\lib\site-packages\keras\engine\training.py", line 1000, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\Alvin Tran\anaconda3\envs\mlenv\lib\site-packages\keras\engine\training.py", line 859, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\Alvin Tran\anaconda3\envs\mlenv\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\Alvin Tran\anaconda3\envs\mlenv\lib\site-packages\keras\engine\input_spec.p

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")